In [1]:
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import time
from functools import reduce

In [2]:
from PISFA import PISFPA

In [3]:
#Error
def _mean_squared_error(y, pred):
    return 0.5 * np.mean((y - pred) ** 2)

def _mean_abs_error(y, pred):
    return np.mean(np.abs(y, pred))

#activate function
def _sigmoid(x):
    return 1. / (1. + np.exp(-x))

def _fourier(x):
    return np.sin(x)

def _identity(x):
    return x

def _hardlimit(x):
    return (x >= 0).astype(int)

#Get function
def getActivation(name):
    return {
        'sigmoid': _sigmoid,
        'fourier': _fourier,
        'hardlimit': _hardlimit,
    }[name]

def getLoss(name):
    return {
        'mse': _mean_squared_error,
        'mae': _mean_abs_error
    }[name]

#T function
def T(x,L1,L2,cn):
    r = x - np.dot(L1,x) + L2
    s = np.abs(r)-cn
    s = np.maximum(s,0,s)
    return s*np.sign(r)

def thetan(x0,x1,n):
    if (x0==x1).all():
        return 0
    else:
        return 1/(2**n*np.linalg.norm(x1-x0,'fro'))

In [4]:
def fit(H, Y, itrs, lam, display_time=False):
    #H = self._activation(X.dot(self._w) + self._bias)

    if display_time:
        start = time.time()

    L = 1. / np.max(np.linalg.eigvals(np.dot(H.T, H))).real
    m = H.shape[1]
    n = Y.shape[1]
    x0 = np.zeros((m,n))
    x1 = np.zeros((m,n))
    L1 = 2*L*np.dot(H.T, H)
    L2 = 2*L*np.dot(H.T, Y)

    for i in range(1,itrs+1):
        cn = ((2e-6*i)/(2*i+1))*lam*L
        beta = 0.9*i/(i+1)
        alpha = 0.9*i/(i+1)

        y = x1 + thetan(x0,x1,i)*(x1-x0)
        z = (1-beta)*x1 + beta*T(x1,L1,L2,cn)

        Ty = T(y,L1,L2,cn)
        Tz = T(z,L1,L2,cn)
        x = (1-alpha)*Ty + alpha*Tz

        x0, x1 = x1, x

    if display_time:
        stop = time.time()
        print(f'Train time: {stop-start}')

    return x

In [5]:
class ELM_AE:
    def __init__(self, n_hidden, X, activation='sigmoid',loss='mse', beta_init=None, w_init=None, bias_init=None ):
        self.X = X

        self._num_input_nodes = X.shape[1]
        self._num_output_units = X.shape[1]
        self._num_hidden_units = n_hidden
        
        self._activation = getActivation(activation)
        self._loss = getLoss(loss)
        
        #weight_out
        if isinstance(beta_init, np.ndarray):
            self._beta = beta_init
        else:
            self._beta = np.random.uniform(-1., 1., (self._num_hidden_units, self._num_output_units))

        #weight_in
        if isinstance(w_init, np.ndarray):
            self._w = w_init
        else:
            self._w = np.random.uniform(-1., 1., (self._num_input_nodes, self._num_hidden_units))

        #bias
        if isinstance(bias_init, np.ndarray):
            self._bias = bias_init
        else:
            self._bias = np.random.uniform(-1., 1., size=(self._num_hidden_units,))
             
        
    def fit(self, itrs, lam, display_time=False):
        H = self._activation(np.dot(self.X, self._w) + self._bias)

        if display_time:
            start = time.time()

        L = 1. / np.max(np.linalg.eigvals(np.dot(H.T, H))).real
        m = H.shape[1]
        n = self._num_output_units
        x0 = np.zeros((m,n))
        x1 = np.zeros((m,n))
        L1 = 2*L*np.dot(H.T, H)
        L2 = 2*L*np.dot(H.T, self.X)

        for i in range(1,itrs+1):
            cn = ((2e-6*i)/(2*i+1))*lam*L
            beta = 0.9*i/(i+1)
            alpha = 0.9*i/(i+1)

            y = x1 + thetan(x0,x1,i)*(x1-x0)
            z = (1-beta)*x1 + beta*T(x1,L1,L2,cn)

            Ty = T(y,L1,L2,cn)
            Tz = T(z,L1,L2,cn)
            x = (1-alpha)*Ty + alpha*Tz

            x0, x1 = x1, x

        if display_time:
            stop = time.time()
            print(f'Train time: {stop-start}')

        self._beta = x
        
    def __call__(self):
        H = self._activation( np.dot(self.X, self._w) + self._bias )
        return np.dot(H, self._beta)

### Prepare Data

In [6]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [7]:
x_train = x_train.reshape((x_train.shape[0], 28 * 28))

In [8]:
x_test = x_test.reshape((x_test.shape[0], 28 * 28))

In [9]:
b = np.zeros((y_train.size, y_train.max()+1))
b[np.arange(y_train.size), y_train] = 1

In [10]:
y_train = b

In [11]:
x_train.shape

(60000, 784)

In [12]:
hidden_node = 100
train_layer = x_train
itrs = 100
lam = 1e-1

In [13]:
model_ae = ELM_AE(hidden_node, train_layer)
model_ae.fit(itrs, lam)

C:\Users\ideapad 330\.conda\envs\AIML\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


In [14]:
beta = model_ae._beta

In [15]:
beta.shape

(100, 784)

In [16]:
bias = model_ae._bias

In [17]:
bias.shape

(100,)

In [18]:
la = model_ae()

C:\Users\ideapad 330\.conda\envs\AIML\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


In [19]:
la.shape

(60000, 784)

In [20]:
def g(w, bias, x):
    activation = getActivation('sigmoid')
    H = activation( np.dot(x, w) + bias )
    return H

In [21]:
h = g(beta.T, bias, la)

C:\Users\ideapad 330\.conda\envs\AIML\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


In [22]:
h.shape

(60000, 100)

In [65]:
hidden_node = [400, 300, 100]
train_layer = x_train
itrs = 600
lam = 1e-1
BETA = []
BIAS = []
for n in hidden_node:
    model_ae = ELM_AE(n, train_layer)
    model_ae.fit(itrs, lam)

    beta = model_ae._beta
    bias = model_ae._bias
    BETA.append(beta.T)
    BIAS.append(bias)
    la = model_ae()
    train_layer = g(beta.T, bias, la)

C:\Users\ideapad 330\.conda\envs\AIML\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


In [66]:
BETA[0].shape

(784, 400)

In [67]:
BETA[1].shape

(400, 300)

In [68]:
X = x_train
for beta, bias in zip(BETA, BIAS):
    train_layer = g(beta, bias, X)
    X = train_layer

C:\Users\ideapad 330\.conda\envs\AIML\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


In [69]:
out_beta = fit(X, y_train, itrs, lam)

In [70]:
pred = X.dot(out_beta)
pred = np.argmax(pred, axis=1)

In [71]:
sum(np.argmax(y_train, axis=1) == pred) / len(y_train)

0.6278666666666667

In [72]:
X = x_test
for beta, bias in zip(BETA, BIAS):
    train_layer = g(beta, bias, X)
    X = train_layer

C:\Users\ideapad 330\.conda\envs\AIML\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


In [73]:
pred = X.dot(out_beta)
pred = np.argmax(pred, axis=1)

In [74]:
sum(y_test == pred) / len(y_test)

0.6347